### ПАКЕТ SURPRISE

* используйте данные MovieLens 1M
* можно использовать любые модели из пакета
* получите RMSE на тестовом сете 0.87 и ниже

#### Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
pip install surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=733561 sha256=c3b432e51e28ac92ac452ab248cde59388b6d2a8f6ef346f836bf2494a186d1d
  Stored in directory: c:\users\solnotka\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [2]:
!surprise

usage: surprise [-h] [-algo <prediction algorithm>]
                [-params <algorithm parameters>]
                [-load-builtin <dataset name>] [-load-custom <file path>]
                [-folds-files <train1 test1 train2 test2... >]
                [-reader <reader>] [-n-folds <number of folds>]
                [-seed <random seed>] [--with-dump] [-dump-dir <dir>]
                [--clean] [-v]

Evaluate the performance of a rating prediction algorithm on a given dataset
using cross validation. You can use a built-in or a custom dataset, and you
can choose to automatically split the dataset into folds, or manually specify
train and test files. Please refer to the documentation page
(http://surprise.readthedocs.io/) for more details.

optional arguments:
  -h, --help            show this help message and exit
  -algo <prediction algorithm>
                        The prediction algorithm to use. Allowed values are
                        NormalPredictor, BaselineOnly, KNNBasic, KNN

error: No algorithm was specified.
